# Hailstone module
Notebook-based [gismo module](https://github.com/ryan-williams/gismo) that computes [the Hailstone sequence](https://en.wikipedia.org/wiki/Collatz_conjecture):

$$
hailstone(x) = 
    \left\{
        \begin{array}{rl}
          3x+1 & \text{if } x \text{ is odd} \\
          x / 2 & \text{otherwise}
        \end{array}
    \right.
$$

This module also demonstrates several features of [gismo modules](https://github.com/ryan-williams/gismo):
- state-propagation across runs
- early-stopping 
- custom commit-messages

## Load current value ("state")
The file `value` will hold an integer; initialize it to `6` if it doesn't exist:

In [ ]:
INITIAL_STATE = 6

from pathlib import Path
value_path = Path('value')
if value_path.exists():
    with value_path.open('r') as f:
        value = int(f.read().strip())
        print('Loaded previous value: %d' % value)
else:
    value = INITIAL_STATE
    print('Set default value: %d' % value)

## Short-circuit if `value` has reached 1

In [ ]:
if value == 1:
    raise Exception('OK: value is 1; exiting early')

## Compute the next value

In [ ]:
prev_value = value
if value % 2 == 0:
    value = value / 2
else:
    value = 3*value + 1
print('New value: %d' % value)

## Write the new value to the "state" path
The [`_STATE`](./_STATE) file in this module is read by the gismo runner, and contains contents:
```
value
```
This tells the runner machinery that any changes to the path `value` should be included in the Git commit representing a run (which is committed in a clone of the module that lives under `runs/` in the module directory). 

Additionally, the runner will merge changes to the `value` file upstream into the outer repo (which subsequent runs are cloned from; note this is not true of other files generated by runs and committed in the `runs/` clone, like the `logs/{out,err}` and `SUCCESS`/`FAILURE` files).

In [ ]:
with value_path.open('w') as f:
    f.write('%d\n' % value)

## Customize each run's commit message
Finally, by writing to the `_MSG`, we can tell gismo what the commit message for this run should be:

In [ ]:
commit_msg_path = Path('_MSG')
with commit_msg_path.open('w') as f:
    f.write('%d → %d\n' % (prev_value, value))